In [1]:
import cv2
import numpy as np
from scipy.signal import medfilt

In [2]:
def edge_diff(frame1, frame2, low_threshold=50, high_threshold=150):
  gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
  gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

  gray1 = cv2.GaussianBlur(gray1, (3,3), 0)
  gray2 = cv2.GaussianBlur(gray2, (3,3), 0)

  edge1 = cv2.Canny(gray1, low_threshold, high_threshold)
  edge2 = cv2.Canny(gray2, low_threshold, high_threshold)

  diff = cv2.absdiff(edge1, edge2)
  return np.mean(diff) / 255.0

In [6]:
def edge_diff(frame1, frame2, blur_ksize=3):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    if blur_ksize > 0:
        gray1 = cv2.GaussianBlur(gray1, (blur_ksize, blur_ksize), 0)
        gray2 = cv2.GaussianBlur(gray2, (blur_ksize, blur_ksize), 0)

    # Адаптивные пороги на основе первого кадра
    median = np.median(gray1)
    low = max(10, int(median * 0.66))
    high = min(245, int(median * 1.33))

    edge1 = cv2.Canny(gray1, low, high)
    edge2 = cv2.Canny(gray2, low, high)

    diff = cv2.absdiff(edge1, edge2)
    return np.sum(diff > 0) / diff.size

In [7]:
def detect_scene(video_path, threshold=2.0, canny_min=50, canny_max=150, smooth_kernel=5):
  cap = cv2.VideoCapture(video_path)
  prive_frame = None
  diffs = []
  fps = cap.get(cv2.CAP_PROP_FPS)

  while cap.isOpened():
    ref, frame = cap.read()
    if not ref:
      break
    if prive_frame is not None:
      d = edge_diff(prive_frame, frame)
      diffs.append(d)
    prive_frame = frame.copy()
  cap.release()

  diffs = np.array(diffs)
  if smooth_kernel > 1 and len(diffs) >= smooth_kernel:
    diffs_smooth = medfilt(diffs, kernel_size=smooth_kernel)
  else:
    diffs_smooth = diffs

  mean = np.mean(diffs_smooth)
  std = np.std(diffs_smooth)
  threshold_new = mean + threshold * std

  scene_change = []
  for i, d, in enumerate(diffs_smooth):
    if d > threshold_new:
      scene_change.append(i + 1 + (smooth_kernel // 2))
  return scene_change, fps

In [8]:
scene_change, fps = detect_scene('/content/drive/MyDrive/practical_data/man_in_costume.mp4', threshold=2.0)
print(f'Кадры смены сцен', scene_change)

Кадры смены сцен [191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247]


In [9]:
len(scene_change)

28

In [ ]:
for frame in scene_change:
  print(f'Кадр {frame} произошел на {(frame / fps):.2f} секунде')

Кадр 191 произошел на 6.37 секунде
Кадр 192 произошел на 6.40 секунде
Кадр 193 произошел на 6.43 секунде
Кадр 194 произошел на 6.47 секунде
Кадр 195 произошел на 6.50 секунде
Кадр 196 произошел на 6.53 секунде
Кадр 197 произошел на 6.57 секунде
Кадр 198 произошел на 6.60 секунде
Кадр 199 произошел на 6.63 секунде
Кадр 200 произошел на 6.67 секунде
Кадр 201 произошел на 6.70 секунде
Кадр 202 произошел на 6.73 секунде
Кадр 203 произошел на 6.77 секунде
Кадр 221 произошел на 7.37 секунде
Кадр 222 произошел на 7.40 секунде
Кадр 223 произошел на 7.43 секунде
Кадр 224 произошел на 7.47 секунде
Кадр 229 произошел на 7.63 секунде
Кадр 230 произошел на 7.67 секунде
Кадр 231 произошел на 7.70 секунде
Кадр 232 произошел на 7.73 секунде
Кадр 233 произошел на 7.77 секунде
Кадр 234 произошел на 7.80 секунде
Кадр 235 произошел на 7.83 секунде
Кадр 236 произошел на 7.87 секунде
Кадр 237 произошел на 7.90 секунде
Кадр 238 произошел на 7.93 секунде
Кадр 239 произошел на 7.97 секунде
Кадр 240 произошел н

In [ ]:
scene_change, fps = detect_scene('/content/drive/MyDrive/practical_data/man_presentation.mp4', threshold=1.0, smooth_kernel=1)
print(f'Кадры смены сцен', scene_change)

Кадры смены сцен [142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 192, 194, 195, 196, 197, 200, 202, 203, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 246, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 338, 343, 348, 353, 358, 361, 363, 368, 373, 378, 383, 388, 393, 395, 396, 397, 398, 399, 400, 404, 407, 40

In [ ]:
len(scene_change)

629